#Topic Modeling (간단한 문서1권)

### 주요 라이브러리 모듈

- gensim: 다양한 텍스트 분석 기능을 제공하는 자연어 처리 패키지
  - gensim.corpora: 고빈도순 어휘 사전 생성
  - gensim.models: LDA/LSA 토픽 모델링
    - 단어의 토픽 가중치, 문서의 토픽 가중치, 토픽의 주요 단어 도출
  - gensim.corpora.CoherenceModel: 토픽 품질 평가


- pyLDAvis
  - 토픽 모델링 결과의 시각화

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Package Import & Parameter Setting

In [2]:
!pip install ujson
!pip install konlpy 
!pip install pyLDAvis 
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 3.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 47.1 MB/s 
     |████████████████████████████████| 453 kB 50.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 27.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=fca5840629864dffe9f865e7b35ea1555a42b66095b7b20b8a17d092ba62d1d6
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
Looking i

## 토픽모델링

In [3]:
# Data Preprocessing Package
import re                       #정규식
import numpy as np
import pandas as pd
import os                      #디렉토리와 경로정보

# NLP Package
from konlpy.tag import * 
import gensim                    #토픽모델링을 하는 라이브러리
import gensim.corpora as corpora #텍스트분석
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package   #LDA시각화
import pyLDAvis 
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


from pprint import pprint #pretty print
import itertools #iterable 객체 처리
import math

import logging #로그처리
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings #경고 무시
warnings.filterwarnings("ignore",category=DeprecationWarning)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


### 데이터 경로 및 저장 경로 설정

In [4]:
# 기본 저장 주소
ROOT_PATH ="/content/gdrive/MyDrive/텍데분/기말과제텍데분/"

# 데이터 저장 주소
DATA_FOLDER_PATH = os.path.join(ROOT_PATH)
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"인공지능youtube댓글.csv")                  # Portal News Crawler에서 저장한 데이터 
DATA_STOP_WORD_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"Data_stop_word.txt")   # 불용어 사전 

# 모델 저장 주소
MODEL_SAVE_FOLDER = os.path.join(ROOT_PATH,"")
LDA_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_lda_topic_modeling.lda")
OPT_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_opt_topic_modeling.lda")

# 결과물 저장 주소
RESULT_FOLDER = os.path.join(ROOT_PATH)
RESULT_SAVE_LDAVIS = os.path.join(RESULT_FOLDER,"Result_lda_vis.html")
RESULT_TOPIC_EXCEL = os.path.join(RESULT_FOLDER,"Result_topic_excel.xlsx")
RESULT_corpus_EXCEL = os.path.join(RESULT_FOLDER,"Result_corpus_excel.xlsx")

###  Data Loading

In [5]:
# 데이터 불러오기 
DF_raw = pd.read_csv('/content/gdrive/MyDrive/텍데분/기말과제텍데분/인공지능youtube댓글.csv') # 엑셀파일의 sheet1시트의 텍스트 가져오기
DF_raw = DF_raw.T.reset_index()

# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',DF_raw.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
DF_raw = DF_raw.dropna(how = 'any') # Null 값이 존재하는 행 제거
DF_raw = DF_raw.drop_duplicates()   # 중복 데이터 프레임 제거 
DF_raw = DF_raw.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', DF_raw.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
print("처리할 데이터수 : ",len(DF_raw))


Null값이 있는지 확인합니다. False

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False

처리할 데이터수 :  4908


In [6]:
# 공백 여러개 한개로 축소
lst = []
for i in DF_raw['index']:
  j = ' '.join(i.split())
  lst.append(j)
df = pd.DataFrame(lst)

# raw데이터로부터 텍스트만 불러오기 
DF_only_text = df[0] #엑셀필드명

#### 토크나이징, 불용어 처리, 말뭉치 생성, 빈도 계수

In [7]:
tokenizer = Okt() # 토큰나이저 지정
stopword_vocab = DATA_STOP_WORD_FILE_NAME # 불용어 파일 불러오기
sep = "\n" # 불용어 처리 인자

def build_vocab(data_frame ,stopword_vocab, separate):
    
    # 불용어 데이터를 가져와 리스트로 변환합니다.
    with open(stopword_vocab, encoding = 'utf-8') as f:
        temp1 = []
        for i in f:
            temp1.append(i)
            
    globals()['stopword_vocab'] = []
    
    # 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
    # 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
    for j in range(len(temp1)):
        temp2 = temp1[j].rstrip(separate)
        globals()['stopword_vocab'].append(temp2)
    
    #okt token에서 명사만 출력합니다. 단어의 길이가 1 초과인 단어만 출력합니다. 
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.nouns(data_frame[i])
        for j in range(len(temp)):

            if len(temp[j]) > 1:
                num_list.append(temp[j])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data =build_vocab(DF_only_text, stopword_vocab, sep)


# 전체 에 대한 워드 카운트 계수 확인

def word_corpus(result_data):
    #전체 단어의 갯수 파악
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    #단어의 빈도수를 확인 후 추가할 불용어 확인 작업
    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 n개의 단어만 저장 vocab
    return vocab

vocab=word_corpus(result_data)

# 전체 워드의 빈도 계수 
df_corpus=pd.DataFrame(columns=["text","count"])
tmp_list=[]
tmp_list1=[]
for word, num in vocab:
    tmp_list.append(word)
    tmp_list1.append(num)
df_corpus['text']=tmp_list
df_corpus['count']=tmp_list1
#상위 20개의 워드 카운드 계수만 출력
a=df_corpus.head(1000)
print(df_corpus.head(20))

#빈도분석한 결과를 별도의 파일에 저장함
a.to_excel(RESULT_corpus_EXCEL ,sheet_name = "sheet1")


# 토픽 모델링 딕셔너리 생성
id2word = corpora.Dictionary(result_data)
 
# 토픽모델링에 사용할 말뭉치 생성
texts = result_data
 
# 용어-문서 빈도
corpus = [id2word.doc2bow(text) for text in texts]

전체 워드의 개수 : 50283
   text  count
0    인간   2422
1    지능   2032
2    인공   1813
3    로봇   1144
4    생각    848
5    사람    663
6    우리    578
7    인류    511
8    영상    382
9    지금    347
10   발전    345
11   기술    272
12   기계    264
13   개발    253
14   미래    251
15   존재    251
16   때문    239
17   지배    232
18   감정    229
19   그냥    227


## 토픽 평가

In [9]:
# Perplexity와 Coherence Score 을 판단
#print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
#print(' ')

#NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
#TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
#save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

TOPICS_W_NUM =20
save_lda_model=0
RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')
for i in range(1,30):
  NUM_TOPICS=i
 
  #해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

  # 토픽 출력
 #  pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
  doc_lda = lda_model[corpus]

  # 모델 저장 
  if save_lda_model == 0:
      lda_model.save(LDA_MODEL_SAVE_NAME)
  # 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)
  """
  해당 셀은 설계한 모델을 계산하는 셀입니다.
  측정은 Perplexity와 Coherence Score입니다.
  """
  #print('\nNUM_TOPICS',NUM_TOPICS)
  # Perplexity 
  #print('Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
  
  # Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  #print('Coherence Score: ', coherence_lda)
 # print('NUM_TOPICS',NUM_TOPICS,'Perplexity: ', lda_model.log_perplexity(corpus),'Coherence: ', coherence_lda)

  print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)

NUM_TOPICS perplexity coherence
T 1 -7.35504200688823 0.42890995669348797
T 2 -7.436803054788707 0.45831673120516414
T 3 -7.483759359050882 0.5101959364791475
T 4 -7.622071253075235 0.45712721885261887
T 5 -7.708812752360198 0.4701796975257852
T 6 -7.76909957434572 0.4753548512571533
T 7 -7.844696674299088 0.450147668785839
T 8 -7.893171430957853 0.44297210775760054
T 9 -7.9525679675308485 0.49178622034786884
T 10 -8.014846039654863 0.48316880811450924
T 11 -8.050217964879208 0.4692001603845932
T 12 -8.09875468966639 0.4888218501755713
T 13 -8.166823816894716 0.4782428972580959
T 14 -8.185924271953334 0.450443512669671
T 15 -8.215582061477244 0.47232773871054395
T 16 -8.248506282299434 0.4850021923601196
T 17 -8.235741272389413 0.478414332688415
T 18 -8.235061379497688 0.4733943654690218
T 19 -8.257809228138349 0.4726656352504689
T 20 -8.254407783191477 0.486917996469925
T 21 -8.266946583165701 0.4653705837066596
T 22 -8.247318668040228 0.505799297003186
T 23 -8.226720771846907 0.51118

In [10]:
print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
print(' ')

NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True

#해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                            num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                            update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                            passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

# 토픽 출력
pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
doc_lda = lda_model[corpus]

# 모델 저장 
if save_lda_model == 0:
    lda_model.save(LDA_MODEL_SAVE_NAME)
# 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)

토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.
 
토픽의 개수를 입력해 주세요. 10
출력할 토픽별 단어의 개수를 입력해 주세요 10
선택한 토픽 모델을 저장하시겠습니까? 
0 저장  
1 미저장  0
[(0,
  '0.093*"인류" + 0.041*"미래" + 0.040*"지구" + 0.029*"통제" + 0.020*"성인" + '
  '0.019*"일자리" + 0.019*"이해" + 0.018*"멸망" + 0.018*"정치" + 0.015*"바로"'),
 (1,
  '0.072*"우리" + 0.048*"기계" + 0.032*"모든" + 0.021*"노동" + 0.021*"수도" + 0.019*"이상" '
  '+ 0.017*"가능성" + 0.017*"현재" + 0.014*"순간" + 0.014*"목적"'),
 (2,
  '0.133*"인간" + 0.114*"지능" + 0.106*"인공" + 0.041*"사람" + 0.041*"생각" + 0.018*"지배" '
  '+ 0.017*"발전" + 0.015*"때문" + 0.015*"지금" + 0.014*"존재"'),
 (3,
  '0.051*"그냥" + 0.030*"대한" + 0.028*"감정" + 0.028*"어간" + 0.019*"선택" + 0.016*"멸종" '
  '+ 0.015*"소름" + 0.014*"실제" + 0.012*"가능" + 0.012*"인식"'),
 (4,
  '0.063*"리뷰" + 0.046*"영화" + 0.041*"기술" + 0.023*"우주" + 0.017*"항상" + '
  '0.015*"하나님" + 0.014*"역사" + 0.014*"컴퓨터" + 0.013*"필요" + 0.013*"상상"'),
 (5,
  '0.017*"경우" + 0.016*"시간" + 0.016*"이유" + 0.016*"자기" + 0.015*"전쟁" + 0.014*"현실" '
  '+ 0.013*"머리" + 0.011*"인정" + 0.011*"유튜버" + 0.011*"세계

In [11]:
#토픽평가
"""
해당 셀은 설계한 모델을 계산하는 셀입니다.
측정은 Perplexity와 Coherence Score입니다.
"""

# Perplexity 
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
# Perplexity는 작을 수록 Coherence Score는 높을 수록 좋다.모델 1개의 값
# 토픽의 개수를 다르게 하여 판단비교해보세요. 동시에 높아지니 믿어도 되겠다. 하나만 선택해라면 코히어런스를 써
# 젠심 코히러런스로 검색해봐서 coherence='c_v'값을 바꿔가면서 해보세요


Perplexity:  -8.014846039654863

Coherence Score:  0.48316880811450924


## 토픽별 키워드 조회

In [12]:
for topic_id in range(NUM_TOPICS):
    topic_word_probs = lda_model.show_topic(topic_id, TOPICS_W_NUM)
    print("Topic ID: {}".format(topic_id))

    for topic_word, prob in topic_word_probs:
        print("\t{}\t{}".format(topic_word, prob))
    print("\n")

Topic ID: 0
	인류	0.09306986629962921
	미래	0.041264209896326065
	지구	0.040372442454099655
	통제	0.02925962768495083
	성인	0.02026754431426525
	일자리	0.01919453591108322
	이해	0.019129598513245583
	멸망	0.018323823809623718
	정치	0.017702670767903328
	바로	0.015266627073287964


Topic ID: 1
	우리	0.07180473953485489
	기계	0.04772644117474556
	모든	0.03188513219356537
	노동	0.02061033435165882
	수도	0.02050262689590454
	이상	0.019054433330893517
	가능성	0.01740073226392269
	현재	0.01720268838107586
	순간	0.014296635054051876
	목적	0.013745257630944252


Topic ID: 2
	인간	0.1331428438425064
	지능	0.11422157287597656
	인공	0.10601508617401123
	사람	0.04132993519306183
	생각	0.04112826660275459
	지배	0.017502959817647934
	발전	0.017072120681405067
	때문	0.015356146730482578
	지금	0.01462211087346077
	존재	0.01353777851909399


Topic ID: 3
	그냥	0.0505078062415123
	대한	0.029581066220998764
	감정	0.028111327439546585
	어간	0.028024805709719658
	선택	0.019038613885641098
	멸종	0.01568235643208027
	소름	0.014841709285974503
	실제	0.01369439996778965
	가능	0.01178752351

## 시각화

In [13]:
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    pyLDAvis.save_html(vis, RESULT_SAVE_LDAVIS)
    return vis
#lda_model or optimal_model
create_vis(lda_model)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.043992  0.098970       1        1   7.994084
1     -0.046998  0.032691       2        1   8.914794
2      0.425397  0.001913       3        1  31.498208
3     -0.055800  0.006861       4        1   5.843324
4     -0.047836  0.035402       5        1   7.677297
5     -0.037501  0.097202       6        1   9.589525
6     -0.050989  0.006294       7        1   6.731836
7     -0.041112 -0.321073       8        1   8.168876
8     -0.046263  0.024370       9        1   6.438187
9     -0.054908  0.017370      10        1   7.143869, topic_info=     Term         Freq        Total Category  logprob  loglift
7      인간  2109.000000  2109.000000  Default  30.0000  30.0000
34     지능  1809.000000  1809.000000  Default  29.0000  29.0000
31     인공  1680.000000  1680.000000  Default  28.0000  28.0000
85     로봇   484.000000   484.000000  Default  27.0000  27.0000
228    인류   375.000000   375.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
81     개미    20.020650    20.948482  Topic10  -5.1897   2.5936
375    행성    19.975505    20.903345  Topic10  -5.1920   2.5935
373    초월    19.846852    20.774753  Topic10  -5.1985   2.5932
1472   신체    18.770389    19.698161  Topic10  -5.2542   2.5907
3161  시냅스    26.047729    27.416419  Topic10  -4.9266   2.5877

[334 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1777      4  0.983843   가능
421       2  0.988189  가능성
154       3  0.991638   가지
585       1  0.943941   간다
255       4  0.974272   감성
...     ...       ...  ...
323       2  0.986616   현재
554      10  0.976313   혹시
42        5  0.976564   확률
710       9  0.972093   환경
166       7  0.949037   휴먼

[309 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

## 결과 저장

In [14]:
# 위에서 선택한 토픽 모델을 엑셀 파일로 떨어뜨리기 위한 함수 
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data):
    sent_topics_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        topics_info_by_doc = row[0]
        topics_info_by_doc = sorted(topics_info_by_doc, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(topics_info_by_doc):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]),
                                                       ignore_index=True)
            else:
                break

    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


'''
확인하고자 하는 모델을 입력하여 데이터 프레임으로 만든 후, 최종 엑셀파일을 저장합니다.
'''

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=result_data)
topic_weight=lda_model[corpus]
df_topic_weight = pd.DataFrame()
for i in range(1, lda_model.num_topics+1):
    df_topic_weight['topic{}'.format(i)]=pd.Series()


# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
 

# LDA모델에서 토픽 웨이트를 추출하여 데이터 프레임에 저장 
for num, topic in enumerate(topic_weight):
    for i in topic[0]:
        df_topic_weight.loc[num, 'topic' + str(i[0]+1)] = i[1]
        #df_topic_weight.loc[num, 'topic' + str(i[0])] = i[1] 
# Null값은 아주 작은 값으로 대체
df_topic_weight = df_topic_weight.fillna(math.exp(-1000)) 

#데이터 프레임 연결
df_topic_last = pd.concat([DF_raw, df_dominant_topic, df_topic_weight], axis=1)
print(df_topic_last)

# 마지막 엑셀 저장
df_topic_last.to_excel(RESULT_TOPIC_EXCEL,sheet_name = "sheet1")

                                                  index  Document_No  \
0     울음터뜨린게 목소리가아닌 인간의 마음을 변용시킬수있는 무기라고 인지하고있다는게 소름...            0   
1     정말 너무도 멋지고 끝내주는 영상들을 만들어주셔서 감사해요 사고와 관점의 지평을 넓...            1   
2     그리고 분과학 내가 본 영상들 중에 최고입니다 한번 보고 반해서 계속 보고 있는데 ...            2   
3     머신러닝 등의 인공지능 교육분야를 다루는 입장에서 한번 진지하게 학생들과 이야기를 ...            3   
4     인공지능의 입장에서 인간 가치관을 질 바라보고 이야기 해주신거 같아요인간을 보살피고...            4   
...                                                 ...          ...   
4903                   ㅅ ㅏ 과 ㅌ ㅣ ㅂ ㅣ로 매 일 내 욕 정을 해결 한 다         4903   
4904                                                알체라         4904   
4905                     사 과 ㅌ ㅣ ㅂ ㅣ보면 휴 가 가 따로 필 요가없 음         4905   
4906                    ㅂ ㅏ 바 ㅌ ㅣ ㅂ ㅣ 지 금 당 장 보러 뛰 어간 다         4906   
4907    성인방송하면ㅅ ㅏ 과 ㅌ ㅣ ㅂ ㅣ아 직 모 르는 사 람 이있 다니 불 쌍 하 다 ㅠ         4907   

      Dominant_Topic  Topic_Perc_Contrib  \
0                2.0              0.2626   
1                2.0              0.2355   
2  